In [9]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm

# Define the range of times from 00 to 12
times = ['00', '01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12']

# Given dates
dates = ["2022-02-24T00:00:00.000000", "2022-02-28T00:00:00.000000", "2022-03-08T00:00:00.000000",
         "2022-03-16T00:00:00.000000", "2022-03-22T00:00:00.000000", "2022-04-05T00:00:00.000000",
         "2022-04-12T00:00:00.000000", "2022-04-20T00:00:00.000000", "2022-04-29T00:00:00.000000",
         "2022-05-03T00:00:00.000000", "2022-05-11T00:00:00.000000", "2022-05-17T00:00:00.000000",
         "2022-05-29T00:00:00.000000"]

# Extract dates without times
dates_without_times = [date.split('T')[0] for date in dates]

# Load PFT dataset
pft_ds = xr.open_dataset('/net/fluo/data3/data/FluoData1/students/renato/aviris_dangermond/California_Vegetation_WHRTYPE_Dangermond/output_latlon.nc')

# Informative message: Loading datasets for all times
print(f'Loading datasets for all times...')

# Load the combined datasets for chl, lai, and lma
chl_ds = xr.open_dataset('/net/fluo/data3/data/FluoData1/students/renato/aviris_dangermond/traits/datasets/clima_fit_prescribed_lai/chl_aviris_dangermond_clima_fit_reg.nc')
lai_ds = xr.open_dataset('/net/fluo/data3/data/FluoData1/students/renato/aviris_dangermond/traits/datasets/clima_fit_prescribed_lai/lwc_aviris_dangermond_clima_fit_reg.nc')
lma_ds = xr.open_dataset('/net/fluo/data3/data/FluoData1/students/renato/aviris_dangermond/traits/datasets/clima_fit_prescribed_lai/lma_aviris_dangermond_clima_fit_reg.nc')

# Extract chl, lai, and lma values and PFT data
chl_values = chl_ds['chl'].values
lai_values = lai_ds['lwc'].values
lma_values = lma_ds['lma'].values * 10000   # Convert lma values to g.m^-2
pft_values = pft_ds['Band1'].values  # Assuming PFT values are stored in Band1 variable

# Define PFT categories (2, 3, 4)
pft_categories = [2, 3, 4]

# Create a mask for PFT categories
pft_mask = np.isin(pft_values, pft_categories)

# Initialize arrays to store new data
new_chl_values = np.zeros_like(chl_values)
new_std_chl_values = np.zeros_like(chl_values)
new_lai_values = np.zeros_like(lai_values)
new_std_lai_values = np.zeros_like(lai_values)
new_lma_values = np.zeros_like(lma_values)
new_std_lma_values = np.zeros_like(lma_values)

# Loop over times
for time_index, time in enumerate(times):
    # Informative message: Loading datasets for the current time
    print(f'Processing data for time: {time}')
    
    # Extract chl, lai, lma values for the current time
    chl_values_time = chl_values[time_index, :, :]
    lai_values_time = lai_values[time_index, :, :]
    lma_values_time = lma_values[time_index, :, :]
    
    # Ensure values and pft_mask have the same shape
    masked_chl_values = np.ma.masked_where(~pft_mask, chl_values_time)
    masked_lai_values = np.ma.masked_where(~pft_mask, lai_values_time)
    masked_lma_values = np.ma.masked_where(~pft_mask, lma_values_time)
    
    # Calculate average and std values per PFT for chl, lai, and lma
    average_chl_per_pft = []
    std_chl_per_pft = []
    average_lai_per_pft = []
    std_lai_per_pft = []
    average_lma_per_pft = []
    std_lma_per_pft = []

    for pft_category in pft_categories:
        # chl
        pft_chl_values = masked_chl_values[pft_values == pft_category]
        average_chl = np.nanmean(pft_chl_values)
        std_chl = np.nanstd(pft_chl_values)
        average_chl_per_pft.append(average_chl)
        std_chl_per_pft.append(std_chl)
        
        # lai
        pft_lai_values = masked_lai_values[pft_values == pft_category]
        average_lai = np.nanmean(pft_lai_values)
        std_lai = np.nanstd(pft_lai_values)
        average_lai_per_pft.append(average_lai)
        std_lai_per_pft.append(std_lai)
        
        # lma
        pft_lma_values = masked_lma_values[pft_values == pft_category]
        pft_lma_values = pft_lma_values[(~np.isnan(pft_lma_values)) & (pft_lma_values != 0)]
        average_lma = np.nanmean(pft_lma_values)
        std_lma = np.nanstd(pft_lma_values)
        average_lma_per_pft.append(average_lma)
        std_lma_per_pft.append(std_lma)
    
    # Print average and standard deviation values per PFT for chl, lai, and lma
    for idx, (avg_chl, std_chl) in enumerate(zip(average_chl_per_pft, std_chl_per_pft)):
        print(f'PFT {pft_categories[idx] -1 }: Average chl value: {avg_chl}, Standard Deviation: {std_chl}')
    for idx, (avg_lai, std_lai) in enumerate(zip(average_lai_per_pft, std_lai_per_pft)):
        print(f'PFT {pft_categories[idx] -1 }: Average lai value: {avg_lai}, Standard Deviation: {std_lai}')
    for idx, (avg_lma, std_lma) in enumerate(zip(average_lma_per_pft, std_lma_per_pft)):
        print(f'PFT {pft_categories[idx]-1}: Average lma value: {avg_lma}, Standard Deviation: {std_lma}')

    # Assign average and std values to new arrays
    for idx, pft_category in enumerate(pft_categories):
        mask = (pft_values == pft_category)
        new_chl_values[time_index, mask] = average_chl_per_pft[idx]
        new_std_chl_values[time_index, mask] = std_chl_per_pft[idx]
        new_lai_values[time_index, mask] = average_lai_per_pft[idx]
        new_std_lai_values[time_index, mask] = std_lai_per_pft[idx]
        new_lma_values[time_index, mask] = average_lma_per_pft[idx]
        new_std_lma_values[time_index, mask] = std_lma_per_pft[idx]

    # Plotting figures for each trait

    # Creating plots for all PFTs for chl
    plt.figure(figsize=(8, 6))  # Adjust figure size
    colors = ['g', 'b', 'r']  # Colors for PFTs 2, 3, 4
    legend_labels = []

    for idx, pft_category in enumerate(pft_categories):
        mask = (pft_values == pft_category)
        
        # Extract chl values for the current PFT category
        chl_for_pft = masked_chl_values[mask].compressed()
        avg_chl = average_chl_per_pft[idx]
        std_chl = std_chl_per_pft[idx]

        # Create a histogram for chl values with transparency
        plt.hist(chl_for_pft, bins=30, density=True, alpha=0.5, color=colors[idx])

        # Add a vertical line for the average
        plt.axvline(avg_chl, color=colors[idx], linestyle='dashed', linewidth=1)

        # Prepare text for the legend
        avg_std_text = f'{avg_chl:.2f} ± {std_chl:.2f}'
        legend_labels.append(f'PFT {pft_category}: {avg_std_text}')

    plt.title(f'Combined CHL Distribution ({dates_without_times[time_index]})', fontsize=18)
    plt.xlabel(r'CHL Value ($\mu$g.cm$^{-2}$)', fontsize=16)
    plt.ylabel('Density', fontsize=16)
    plt.ylim(0, 0.3)  # Set y-axis limits
    plt.xlim(0, 90)  # Set y-axis limits
    plt.legend(legend_labels, fontsize=14)  # Increase legend fontsize
    plt.xticks(fontsize=14)  # Increase x-axis tick label fontsize
    plt.yticks(fontsize=14)  # Increase y-axis tick label fontsize

    # Save the combined plot as a PNG file
    plt.savefig(f'/net/fluo/data3/data/FluoData1/students/renato/aviris_dangermond/traits/figures/clima_fit_prescribed_lai/combined_chl_histogram_time_{time}.png', dpi=300)
    plt.close()

    # Creating plots for all PFTs for lai
    plt.figure(figsize=(8, 6))  # Adjust figure size
    colors = ['g', 'b', 'r']  # Colors for PFTs 2, 3, 4
    legend_labels = []

    for idx, pft_category in enumerate(pft_categories):
        mask = (pft_values == pft_category)
        
        # Extract lai values for the current PFT category
        lai_for_pft = masked_lai_values[mask].compressed()
        avg_lai = average_lai_per_pft[idx]
        std_lai = std_lai_per_pft[idx]

        # Create a histogram for lai values with transparency
        plt.hist(lai_for_pft, bins=30, density=True, alpha=0.5, color=colors[idx])

        # Add a vertical line for the average
        plt.axvline(avg_lai, color=colors[idx], linestyle='dashed', linewidth=1)

        # Prepare text for the legend
        avg_std_text = f'{avg_lai:.2f} ± {std_lai:.2f}'
        legend_labels.append(f'PFT {pft_category}: {avg_std_text}')

    plt.title(f'Combined LWC Distribution ({dates_without_times[time_index]})', fontsize=18)
    plt.xlabel(r'LWC Value (mol.m$^{-2}$)', fontsize=16)
    plt.ylabel('Density', fontsize=16)
    plt.ylim(0, 1.0)  # Set y-axis limits
    plt.xlim(0, 20)  # Set y-axis limits
    plt.legend(legend_labels, fontsize=14)  # Increase legend fontsize
    plt.xticks(fontsize=14)  # Increase x-axis tick label fontsize
    plt.yticks(fontsize=14)  # Increase y-axis tick label fontsize

    # Save the combined plot as a PNG file
    plt.savefig(f'/net/fluo/data3/data/FluoData1/students/renato/aviris_dangermond/traits/figures/clima_fit_prescribed_lai/combined_lwc_histogram_time_{time}.png')
    plt.close()

    # Creating plots for all PFTs for lma
    plt.figure(figsize=(8, 6))  # Adjust figure size
    colors = ['g', 'b', 'r']  # Colors for PFTs 2, 3, 4
    legend_labels = []

    for idx, pft_category in enumerate(pft_categories):
        mask = (pft_values == pft_category)
        
        # Extract lma values for the current PFT category
        lma_for_pft = masked_lma_values[mask].compressed()
        avg_lma = average_lma_per_pft[idx]
        std_lma = std_lma_per_pft[idx]

        # Create a histogram for lma values with transparency
        plt.hist(lma_for_pft, bins=30, density=True, alpha=0.5, color=colors[idx])

        # Add a vertical line for the average
        plt.axvline(avg_lma, color=colors[idx], linestyle='dashed', linewidth=1)

        # Prepare text for the legend
        avg_std_text = f'{avg_lma:.2e} ± {std_lma:.2e}'
        legend_labels.append(f'PFT {pft_category}: {avg_std_text}')

    plt.title(f'Combined LMA Distribution ({dates_without_times[time_index]})', fontsize=18)
    plt.xlabel(r'LMA Value (g.cm$^{-2}$)', fontsize=16)
    plt.ylabel('Density', fontsize=16)
    plt.ylim(0, 1.0)  # Set y-axis limits
    plt.xlim(0, 500)  # Set y-axis limits
    plt.legend(legend_labels, fontsize=14)  # Increase legend fontsize
    plt.xticks(fontsize=14)  # Increase x-axis tick label fontsize
    plt.yticks(fontsize=14)  # Increase y-axis tick label fontsize

    # Save the combined plot as a PNG file
    plt.savefig(f'/net/fluo/data3/data/FluoData1/students/renato/aviris_dangermond/traits/figures/clima_fit_prescribed_lai/combined_lma_histogram_time_{time}.png')
    plt.close()

# Replace zeros by NaN in new maps
new_chl_values = np.where(new_chl_values != 0, new_chl_values, np.nan)
new_std_chl_values = np.where(new_std_chl_values != 0, new_std_chl_values, np.nan)
new_lai_values = np.where(new_lai_values != 0, new_lai_values, np.nan)
new_std_lai_values = np.where(new_std_lai_values != 0, new_std_lai_values, np.nan)
new_lma_values = np.where(new_lma_values != 0, new_lma_values, np.nan)
new_std_lma_values = np.where(new_std_lma_values != 0, new_std_lma_values, np.nan)

# Save the new datasets to NetCDF files
new_chl_ds = xr.Dataset({
    'chl': (('time', 'lat', 'lon'), new_chl_values),
    'std_chl': (('time', 'lat', 'lon'), new_std_chl_values)},
    coords={'time': chl_ds['time'], 'lat': chl_ds['lat'], 'lon': chl_ds['lon']})
new_chl_ds.to_netcdf(f'/net/fluo/data3/data/FluoData1/students/renato/aviris_dangermond/traits/datasets/clima_fit_prescribed_lai/masked_chl_aviris_dangermond_clima_fit.nc')

new_lai_ds = xr.Dataset({
    'lwc': (('time', 'lat', 'lon'), new_lai_values),
    'std_lwc': (('time', 'lat', 'lon'), new_std_lai_values)},
    coords={'time': lai_ds['time'], 'lat': lai_ds['lat'], 'lon': lai_ds['lon']})
new_lai_ds.to_netcdf(f'/net/fluo/data3/data/FluoData1/students/renato/aviris_dangermond/traits/datasets/clima_fit_prescribed_lai/masked_lwc_aviris_dangermond_clima_fit.nc')

new_lma_ds = xr.Dataset({
    'lma': (('time', 'lat', 'lon'), new_lma_values),
    'std_lma': (('time', 'lat', 'lon'), new_std_lma_values)},
    coords={'time': lma_ds['time'], 'lat': lma_ds['lat'], 'lon': lma_ds['lon']})
new_lma_ds.to_netcdf(f'/net/fluo/data3/data/FluoData1/students/renato/aviris_dangermond/traits/datasets/clima_fit_prescribed_lai/masked_lma_aviris_dangermond_clima_fit.nc')

# Informative message: Data processing and saving completed
print(f'Data processing and saving completed.\n')

# Close the opened datasets
chl_ds.close()
lai_ds.close()
lma_ds.close()
pft_ds.close()


Loading datasets for all times...
Processing data for time: 00
PFT 1: Average chl value: 63.857608795166016, Standard Deviation: 26.526578903198242
PFT 2: Average chl value: 58.02014923095703, Standard Deviation: 26.676130294799805
PFT 3: Average chl value: 76.7792739868164, Standard Deviation: 20.860881805419922
PFT 1: Average lai value: 4.790730953216553, Standard Deviation: 5.423482894897461
PFT 2: Average lai value: 4.412163257598877, Standard Deviation: 5.146036148071289
PFT 3: Average lai value: 7.9902825355529785, Standard Deviation: 6.627671718597412
PFT 1: Average lma value: 113.1168441772461, Standard Deviation: 120.94055938720703
PFT 2: Average lma value: 99.09156036376953, Standard Deviation: 113.34367370605469
PFT 3: Average lma value: 191.34396362304688, Standard Deviation: 149.70326232910156
Processing data for time: 01
PFT 1: Average chl value: 60.826168060302734, Standard Deviation: 27.247961044311523
PFT 2: Average chl value: 55.58740997314453, Standard Deviation: 26.

In [4]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm

# Define the range of times from 00 to 12
times = ['00', '01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12']

# Given dates
dates = ["2022-02-24T00:00:00.000000", "2022-02-28T00:00:00.000000", "2022-03-08T00:00:00.000000",
         "2022-03-16T00:00:00.000000", "2022-03-22T00:00:00.000000", "2022-04-05T00:00:00.000000",
         "2022-04-12T00:00:00.000000", "2022-04-20T00:00:00.000000", "2022-04-29T00:00:00.000000",
         "2022-05-03T00:00:00.000000", "2022-05-11T00:00:00.000000", "2022-05-17T00:00:00.000000",
         "2022-05-29T00:00:00.000000"]

# Extract dates without times
dates_without_times = [date.split('T')[0] for date in dates]

# Loop over times
for time in times:

    # Informative message: Loading datasets for the current time
    print(f'Processing data for time: {time}')
    
    # Load chl and PFT datasets for the current time
    chl_ds = xr.open_dataset(f'/net/fluo/data3/data/FluoData1/students/renato/aviris_dangermond/traits/datasets/clima_fit_prescribed_lai/chl_aviris_dangermond_clima_fit_time_{time}_reg.nc')
    lai_ds = xr.open_dataset(f'/net/fluo/data3/data/FluoData1/students/renato/aviris_dangermond/traits/datasets/clima_fit_prescribed_lai/lwc_aviris_dangermond_clima_fit_time_{time}_reg.nc')
    lma_ds = xr.open_dataset(f'/net/fluo/data3/data/FluoData1/students/renato/aviris_dangermond/traits/datasets/clima_fit_prescribed_lai/lma_aviris_dangermond_clima_fit_time_{time}_reg.nc')
    pft_ds = xr.open_dataset(f'../California_Vegetation_WHRTYPE_Dangermond/output_latlon.nc')  # Update the PFT dataset filename
    
    # Informative message: Extracting data from the loaded datasets
    print('Extracting data from the loaded datasets...')
    
    # Extract chl values and PFT data
    chl_values = chl_ds['chl'].values
    lai_values = lai_ds['lwc'].values
    lma_values = lma_ds['lma'].values
    pft_values = pft_ds['Band1'].values  # Assuming PFT values are stored in Band1 variable
    
    # Define PFT categories (2, 3, 4)
    pft_categories = [2, 3, 4]
    
    # Create a mask for PFT categories
    pft_mask = np.isin(pft_values, pft_categories)
    
    # Ensure chl_values and pft_mask have the same shape
    masked_chl_values = np.ma.masked_where(~pft_mask[:, :], chl_values[0, :, :])
    masked_lai_values = np.ma.masked_where(~pft_mask[:, :], lai_values[0, :, :])
    masked_lma_values = np.ma.masked_where(~pft_mask[:, :], lma_values[0, :, :])
    
    # Calculate average chl value per PFT
    average_chl_per_pft = []
    std_chl_per_pft = []
    average_lai_per_pft = []
    std_lai_per_pft = []
    average_lma_per_pft = []
    std_lma_per_pft = []
    for pft_category in pft_categories:
        # chl
        pft_chl_values = masked_chl_values[pft_values[:, :] == pft_category]
        average_chl = np.nanmean(pft_chl_values)
        std_chl = np.nanstd(pft_chl_values)
        average_chl_per_pft.append(average_chl)
        std_chl_per_pft.append(std_chl)
        
        # lai
        pft_lai_values = masked_lai_values[pft_values[:, :] == pft_category]
        average_lai = np.nanmean(pft_lai_values)
        std_lai = np.nanstd(pft_lai_values)
        average_lai_per_pft.append(average_lai)
        std_lai_per_pft.append(std_lai)
        
        # lma
        pft_lma_values = masked_lma_values[pft_values[:, :] == pft_category]
        pft_lma_values = pft_lma_values[(~np.isnan(pft_lma_values)) & (pft_lma_values != 0)]
        average_lma = np.nanmean(pft_lma_values)
        std_lma = np.nanstd(pft_lma_values)
        average_lma_per_pft.append(average_lma)
        std_lma_per_pft.append(std_lma)
    
    # Print average and standard deviation chl values per PFT
    for idx, (avg_chl, std_chl) in enumerate(zip(average_chl_per_pft, std_chl_per_pft)):
        print(f'PFT {pft_categories[idx] -1 }: Average chl value: {avg_chl}, Standard Deviation: {std_chl}')

    # Print average and standard deviation lai values per PFT
    for idx, (avg_lai, std_lai) in enumerate(zip(average_lai_per_pft, std_lai_per_pft)):
        print(f'PFT {pft_categories[idx] -1 }: Average lai value: {avg_lai}, Standard Deviation: {std_lai}')

    # Print average and standard deviation lma values per PFT
    for idx, (avg_lma, std_lma) in enumerate(zip(average_lma_per_pft, std_lma_per_pft)):
        print(f'PFT {pft_categories[idx]-1}: Average lma value: {avg_lma}, Standard Deviation: {std_lma}')

    # Optionally, save the new chl map with masked values
    # chl_ds['chl'][:] = masked_chl_values
    # chl_ds.to_netcdf('masked_chl_aviris_dangermond.nc')

    # Create a new chl map with average values per PFT
    new_chl_values = np.zeros_like(chl_values[0,:,:])
    new_std_chl_values = np.zeros_like(chl_values[0,:,:])
    new_lai_values = np.zeros_like(lai_values[0,:,:])
    new_std_lai_values = np.zeros_like(lai_values[0,:,:])
    new_lma_values = np.zeros_like(lma_values[0,:,:])
    new_std_lma_values = np.zeros_like(lma_values[0,:,:])


    # Creating plots for all PFTs for chl
    #plt.figure()
    plt.figure(figsize=(8, 6))  # Adjust figure size
    colors = ['g', 'b', 'r']  # Colors for PFTs 2, 3, 4
    legend_labels = []

    for idx, pft_category in enumerate(pft_categories):
        mask = (pft_values == pft_category)[:, :]
        new_chl_values[mask] = average_chl_per_pft[idx]
        new_std_chl_values[mask] = std_chl_per_pft[idx]
        #new_lai_values[mask] = average_lai_per_pft[idx]
        #new_std_lai_values[mask] = std_lai_per_pft[idx]
        #new_lma_values[mask] = average_lma_per_pft[idx]
        #new_std_lma_values[mask] = std_lma_per_pft[idx]

        # Extract chl values for the current PFT category
        chl_for_pft = masked_chl_values[pft_values == pft_category].compressed()
        avg_chl = average_chl_per_pft[idx]
        std_chl = std_chl_per_pft[idx]

        # Create a histogram for chl values with transparency
        plt.hist(chl_for_pft, bins=30, density=True, alpha=0.5, color=colors[idx])

        # Add a vertical line for the average
        plt.axvline(avg_chl, color=colors[idx], linestyle='dashed', linewidth=1)

        # Prepare text for the legend
        avg_std_text = f'{avg_chl:.2f} ± {std_chl:.2f}'
        legend_labels.append(f'PFT {pft_category}: {avg_std_text}')

    plt.title(f'Combined CHL Distribution ({dates_without_times[int(time)]})', fontsize=18)
    plt.xlabel(r'CHL Value ($\mu$g.cm$^{-2}$)', fontsize=16)
    plt.ylabel('Density', fontsize=16)
    plt.ylim(0, 0.5)  # Set y-axis limits
    plt.xlim(0, 20)  # Set y-axis limits
    plt.legend(legend_labels, fontsize=14)  # Increase legend fontsize
    plt.xticks(fontsize=14)  # Increase x-axis tick label fontsize
    plt.yticks(fontsize=14)  # Increase y-axis tick label fontsize

    # Save the combined plot as a PNG file
    #plt.savefig(f'combined_chl_histogram_time_{time}.png',dpi=300)
    plt.close()

    # Creating plots for all PFTs for lai
    plt.figure(figsize=(8, 6))  # Adjust figure size
    colors = ['g', 'b', 'r']  # Colors for PFTs 2, 3, 4
    legend_labels = []

    for idx, pft_category in enumerate(pft_categories):
        mask = (pft_values == pft_category)[:, :]
        #new_chl_values[mask] = average_chl_per_pft[idx]
        #new_std_chl_values[mask] = std_chl_per_pft[idx]
        new_lai_values[mask] = average_lai_per_pft[idx]
        new_std_lai_values[mask] = std_lai_per_pft[idx]
        #new_lma_values[mask] = average_lma_per_pft[idx]
        #new_std_lma_values[mask] = std_lma_per_pft[idx]

        # Extract chl values for the current PFT category
        lai_for_pft = masked_lai_values[pft_values == pft_category].compressed()
        avg_lai = average_lai_per_pft[idx]
        std_lai = std_lai_per_pft[idx]

        # Create a histogram for chl values with transparency
        plt.hist(lai_for_pft, bins=30, density=True, alpha=0.5, color=colors[idx])

        # Add a vertical line for the average
        plt.axvline(avg_lai, color=colors[idx], linestyle='dashed', linewidth=1)

        # Prepare text for the legend
        avg_std_text = f'{avg_lai:.2f} ± {std_lai:.2f}'
        legend_labels.append(f'PFT {pft_category}: {avg_std_text}')

    plt.title(f'Combined LAI Distribution ({dates_without_times[int(time)]})', fontsize=18)
    plt.xlabel(r'LAI Value (m$^{2}$.m$^{-2}$)', fontsize=16)
    plt.ylabel('Density', fontsize=16)
    plt.ylim(0, 1.0)  # Set y-axis limits
    plt.xlim(0, 7.5)  # Set y-axis limits
    plt.legend(legend_labels, fontsize=14)  # Increase legend fontsize
    plt.xticks(fontsize=14)  # Increase x-axis tick label fontsize
    plt.yticks(fontsize=14)  # Increase y-axis tick label fontsize

    # Save the combined plot as a PNG file
    #plt.savefig(f'combined_lai_histogram_time_{time}.png')
    plt.close()

    # Creating plots for all PFTs for lma
    plt.figure(figsize=(8, 6))  # Adjust figure size
    colors = ['g', 'b', 'r']  # Colors for PFTs 2, 3, 4
    legend_labels = []

    for idx, pft_category in enumerate(pft_categories):
        mask = (pft_values == pft_category)[:, :]
        #new_chl_values[mask] = average_chl_per_pft[idx]
        #new_std_chl_values[mask] = std_chl_per_pft[idx]
        #new_lai_values[mask] = average_lai_per_pft[idx]
        #new_std_lai_values[mask] = std_lai_per_pft[idx]
        new_lma_values[mask] = average_lma_per_pft[idx]
        new_std_lma_values[mask] = std_lma_per_pft[idx]

        # Extract chl values for the current PFT category
        lma_for_pft = masked_lma_values[pft_values == pft_category].compressed()
        avg_lma = average_lma_per_pft[idx]
        std_lma = std_lma_per_pft[idx]

        # Create a histogram for chl values with transparency
        plt.hist(lma_for_pft, bins=30, density=True, alpha=0.5, color=colors[idx])

        # Add a vertical line for the average
        plt.axvline(avg_lma, color=colors[idx], linestyle='dashed', linewidth=1)

        # Prepare text for the legend
        avg_std_text = f'{avg_lma:.2e} ± {std_lma:.2e}'
        legend_labels.append(f'PFT {pft_category}: {avg_std_text}')

    print(np.min(lma_for_pft),np.max(lma_for_pft))
    plt.title(f'Combined LMA Distribution ({dates_without_times[int(time)]})', fontsize=18)
    plt.xlabel(r'LMA Value (g.cm$^{-2}$)', fontsize=16)
    plt.ylabel('Density', fontsize=16)
    plt.ylim(0, 800)  # Set y-axis limits
    plt.xlim(0, 0.03)  # Set y-axis limits
    plt.legend(legend_labels, fontsize=14)  # Increase legend fontsize
    plt.xticks(fontsize=14)  # Increase x-axis tick label fontsize
    plt.yticks(fontsize=14)  # Increase y-axis tick label fontsize

    # Save the combined plot as a PNG file
    #plt.savefig(f'combined_lma_histogram_time_{time}.png')
    plt.close()

    

    # Create new xarray DataArrays for chl and std
    new_chl_da = xr.DataArray(new_chl_values[np.newaxis,:,:], dims=('time', 'lat', 'lon'), coords={'time': chl_ds['time'], 'lat': chl_ds['lat'], 'lon': chl_ds['lon']})
    new_std_chl_da = xr.DataArray(new_std_chl_values[np.newaxis,:,:], dims=('time', 'lat', 'lon'), coords={'time': chl_ds['time'], 'lat': chl_ds['lat'], 'lon': chl_ds['lon']})


    # Replace zeros by NaN in new maps
    new_chl_da = new_chl_da.where(new_chl_da != 0, np.nan)
    new_std_chl_da = new_std_chl_da.where(new_std_chl_da != 0, np.nan)

    # Create a new xarray Dataset with chl and std DataArrays
    new_chl_data = {'chl': new_chl_da, 'std': new_std_chl_da}
    new_chl_ds = xr.Dataset(new_chl_data)
    

    # Save the new chl map with masked values and standard deviation to a NetCDF file
    new_chl_ds.to_netcdf(f'/net/fluo/data3/data/FluoData1/students/renato/aviris_dangermond/traits/datasets/clima_fit_prescribed_lai/masked_chl_aviris_dangermond_time_{time}.nc')

    print(lai_values.shape)
    print(new_lai_values.shape)
    # Create new xarray DataArrays for lai and std
    #new_lai_da = xr.DataArray(new_lai_values[np.newaxis,:,:], dims=('time', 'lat', 'lon'), coords={'time': lai_ds['time'], 'lat': lai_ds['lat'], 'lon': lai_ds['lon']})
    lai_values = np.clip(lai_values, 1e-1, 20.)  # Clip values between 0.1 and max_lai
    new_lai_values = lai_values[0,:,:]*(new_std_lai_values/new_std_lai_values)
    
    new_lai_da = xr.DataArray(new_lai_values[np.newaxis,:,:], dims=('time', 'lat', 'lon'), coords={'time': lai_ds['time'], 'lat': lai_ds['lat'], 'lon': lai_ds['lon']})
    new_std_lai_da = xr.DataArray(new_std_lai_values[np.newaxis,:,:], dims=('time', 'lat', 'lon'), coords={'time': lai_ds['time'], 'lat': lai_ds['lat'], 'lon': lai_ds['lon']})
    
    # Replace zeros by NaN in new maps
    new_lai_da = new_lai_da.where(new_lai_da != 0, np.nan)
    new_lai_da = new_lai_da.where(~np.isnan(new_lai_da), np.nan)

    new_std_lai_da = new_std_lai_da.where(new_std_lai_da != 0, np.nan)

    # Create a new xarray Dataset with lai and std DataArrays
    new_lai_data = {'lwc': new_lai_da, 'std': new_std_lai_da}
    new_lai_ds = xr.Dataset(new_lai_data)
    
    # Save the new lai map with masked values and standard deviation to a NetCDF file
    new_lai_ds.to_netcdf(f'/net/fluo/data3/data/FluoData1/students/renato/aviris_dangermond/traits/datasets/clima_fit_prescribed_lai/masked_lwc_aviris_dangermond_time_{time}.nc')

    # Create new xarray DataArrays for lma and std
    new_lma_da = xr.DataArray(new_lma_values[np.newaxis,:,:], dims=('time', 'lat', 'lon'), coords={'time': lma_ds['time'], 'lat': lma_ds['lat'], 'lon': lma_ds['lon']})
    new_std_lma_da = xr.DataArray(new_std_lma_values[np.newaxis,:,:], dims=('time', 'lat', 'lon'), coords={'time': lma_ds['time'], 'lat': lma_ds['lat'], 'lon': lma_ds['lon']})
    
    new_lma_da = new_lma_da.where(new_lma_da != 0, np.nan)
    new_std_lma_da = new_std_lma_da.where(new_std_lma_da != 0, np.nan)

    # Create a new xarray Dataset with lma and std DataArrays
    new_lma_data = {'lma': new_lma_da, 'std': new_std_lma_da}
    new_lma_ds = xr.Dataset(new_lma_data)
    


    # Save the new lma map with masked values and standard deviation to a NetCDF file
    new_lma_ds.to_netcdf(f'/net/fluo/data3/data/FluoData1/students/renato/aviris_dangermond/traits/datasets/clima_fit_prescribed_lai/masked_lma_aviris_dangermond_time_{time}.nc')
    
    
    # Informative message: Data processing and saving completed for the current time
    print(f'Data processing and saving completed for time: {time}\n')

    # Close the opened datasets
    chl_ds.close()
    lai_ds.close()
    lma_ds.close()
    pft_ds.close()



Processing data for time: 00
Extracting data from the loaded datasets...
PFT 1: Average chl value: 63.857608795166016, Standard Deviation: 26.526578903198242
PFT 2: Average chl value: 58.02014923095703, Standard Deviation: 26.676130294799805
PFT 3: Average chl value: 76.7792739868164, Standard Deviation: 20.860881805419922
PFT 1: Average lai value: 4.790730953216553, Standard Deviation: 5.423482894897461
PFT 2: Average lai value: 4.412163257598877, Standard Deviation: 5.146036148071289
PFT 3: Average lai value: 7.9902825355529785, Standard Deviation: 6.627671718597412
PFT 1: Average lma value: 0.011311683803796768, Standard Deviation: 0.012094056233763695
PFT 2: Average lma value: 0.009909155778586864, Standard Deviation: 0.011334367096424103
PFT 3: Average lma value: 0.01913439668715, Standard Deviation: 0.014970325864851475
nan nan
(1, 458, 492)
(458, 492)
Data processing and saving completed for time: 00

Processing data for time: 01


/tmp/ipykernel_578349/1655304533.py:258: RuntimeWarning: invalid value encountered in divide
  new_lai_values = lai_values[0,:,:]*(new_std_lai_values/new_std_lai_values)


Extracting data from the loaded datasets...
PFT 1: Average chl value: 60.826168060302734, Standard Deviation: 27.247961044311523
PFT 2: Average chl value: 55.58740997314453, Standard Deviation: 26.950624465942383
PFT 3: Average chl value: 75.41986846923828, Standard Deviation: 21.62092399597168
PFT 1: Average lai value: 4.298801422119141, Standard Deviation: 5.073195457458496
PFT 2: Average lai value: 3.93855357170105, Standard Deviation: 4.786709308624268
PFT 3: Average lai value: 7.37850284576416, Standard Deviation: 6.30325984954834
PFT 1: Average lma value: 0.009866194799542427, Standard Deviation: 0.01107361912727356
PFT 2: Average lma value: 0.008575400337576866, Standard Deviation: 0.010338558815419674
PFT 3: Average lma value: 0.017166931182146072, Standard Deviation: 0.014000636525452137
nan nan
(1, 458, 492)
(458, 492)
Data processing and saving completed for time: 01

Processing data for time: 02


/tmp/ipykernel_578349/1655304533.py:258: RuntimeWarning: invalid value encountered in divide
  new_lai_values = lai_values[0,:,:]*(new_std_lai_values/new_std_lai_values)


Extracting data from the loaded datasets...
PFT 1: Average chl value: 62.032020568847656, Standard Deviation: 26.74464225769043
PFT 2: Average chl value: 55.34238052368164, Standard Deviation: 26.542369842529297
PFT 3: Average chl value: 75.20604705810547, Standard Deviation: 21.22895050048828
PFT 1: Average lai value: 4.440042495727539, Standard Deviation: 5.2257771492004395
PFT 2: Average lai value: 3.9477312564849854, Standard Deviation: 4.867476940155029
PFT 3: Average lai value: 7.55593204498291, Standard Deviation: 6.323657512664795
PFT 1: Average lma value: 0.010244429111480713, Standard Deviation: 0.010856199078261852
PFT 2: Average lma value: 0.008803905919194221, Standard Deviation: 0.009909162297844887
PFT 3: Average lma value: 0.01679212786257267, Standard Deviation: 0.013240792788565159
nan nan
(1, 458, 492)
(458, 492)
Data processing and saving completed for time: 02

Processing data for time: 03


...

/tmp/ipykernel_578349/1655304533.py:258: RuntimeWarning: invalid value encountered in divide
  new_lai_values = lai_values[0,:,:]*(new_std_lai_values/new_std_lai_values)


Extracting data from the loaded datasets...
PFT 1: Average chl value: 23.766983032226562, Standard Deviation: 21.47966766357422
PFT 2: Average chl value: 17.296283721923828, Standard Deviation: 17.924806594848633
PFT 3: Average chl value: 39.0125846862793, Standard Deviation: 26.075592041015625
PFT 1: Average lai value: 1.0595940351486206, Standard Deviation: 1.8095520734786987
PFT 2: Average lai value: 0.835856020450592, Standard Deviation: 1.7098736763000488
PFT 3: Average lai value: 2.448489189147949, Standard Deviation: 2.591031312942505
PFT 1: Average lma value: 0.0035443950910121202, Standard Deviation: 0.0044942572712898254
PFT 2: Average lma value: 0.002897122409194708, Standard Deviation: 0.004082067403942347
PFT 3: Average lma value: 0.007134462706744671, Standard Deviation: 0.0058570136316120625
nan nan
(1, 458, 492)
(458, 492)
Data processing and saving completed for time: 11

Processing data for time: 12


/tmp/ipykernel_578349/1655304533.py:258: RuntimeWarning: invalid value encountered in divide
  new_lai_values = lai_values[0,:,:]*(new_std_lai_values/new_std_lai_values)


Extracting data from the loaded datasets...
PFT 1: Average chl value: 35.14242935180664, Standard Deviation: 26.68703269958496
PFT 2: Average chl value: 27.29800796508789, Standard Deviation: 24.78179931640625
PFT 3: Average chl value: 50.00579833984375, Standard Deviation: 26.72353744506836
PFT 1: Average lai value: 1.298640489578247, Standard Deviation: 2.1907765865325928
PFT 2: Average lai value: 1.0639183521270752, Standard Deviation: 2.236875057220459
PFT 3: Average lai value: 2.9480679035186768, Standard Deviation: 3.065063953399658
PFT 1: Average lma value: 0.004328400362282991, Standard Deviation: 0.005127462558448315
PFT 2: Average lma value: 0.003487904090434313, Standard Deviation: 0.004736403003334999
PFT 3: Average lma value: 0.008305491879582405, Standard Deviation: 0.006376227829605341
nan nan
(1, 458, 492)
(458, 492)
Data processing and saving completed for time: 12



/tmp/ipykernel_578349/1655304533.py:258: RuntimeWarning: invalid value encountered in divide
  new_lai_values = lai_values[0,:,:]*(new_std_lai_values/new_std_lai_values)


### Generate PFT average files separate by time

In [6]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm

# Define the range of times from 00 to 12
times = ['00', '01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12']

# Given dates
dates = ["2022-02-24T00:00:00.000000", "2022-02-28T00:00:00.000000", "2022-03-08T00:00:00.000000",
         "2022-03-16T00:00:00.000000", "2022-03-22T00:00:00.000000", "2022-04-05T00:00:00.000000",
         "2022-04-12T00:00:00.000000", "2022-04-20T00:00:00.000000", "2022-04-29T00:00:00.000000",
         "2022-05-03T00:00:00.000000", "2022-05-11T00:00:00.000000", "2022-05-17T00:00:00.000000",
         "2022-05-29T00:00:00.000000"]

# Extract dates without times
dates_without_times = [date.split('T')[0] for date in dates]

# Loop over times
for time in times:

    # Informative message: Loading datasets for the current time
    print(f'Processing data for time: {time}')
    
    # Load chl and PFT datasets for the current time
    chl_ds = xr.open_dataset(f'/net/fluo/data3/data/FluoData1/students/renato/aviris_dangermond/traits/datasets/clima_fit_prescribed_lai/chl_aviris_dangermond_clima_fit_time_{time}_reg.nc')
    lai_ds = xr.open_dataset(f'/net/fluo/data3/data/FluoData1/students/renato/aviris_dangermond/traits/datasets/clima_fit_prescribed_lai/lwc_aviris_dangermond_clima_fit_time_{time}_reg.nc')
    lma_ds = xr.open_dataset(f'/net/fluo/data3/data/FluoData1/students/renato/aviris_dangermond/traits/datasets/clima_fit_prescribed_lai/lma_aviris_dangermond_clima_fit_time_{time}_reg.nc')
    pft_ds = xr.open_dataset(f'../California_Vegetation_WHRTYPE_Dangermond/output_latlon.nc')  # Update the PFT dataset filename
    
    # Informative message: Extracting data from the loaded datasets
    print('Extracting data from the loaded datasets...')
    
    # Extract chl values and PFT data
    chl_values = chl_ds['chl'].values
    lai_values = lai_ds['lwc'].values
    lma_values = lma_ds['lma'].values
    pft_values = pft_ds['Band1'].values  # Assuming PFT values are stored in Band1 variable
    
    # Define PFT categories (2, 3, 4)
    pft_categories = [2, 3, 4]
    
    # Create a mask for PFT categories
    pft_mask = np.isin(pft_values, pft_categories)
    
    # Ensure chl_values, lai_values, lma_values, and pft_mask have the same shape
    masked_chl_values = np.ma.masked_where(~pft_mask[:, :], chl_values[0, :, :])
    masked_lai_values = np.ma.masked_where(~pft_mask[:, :], lai_values[0, :, :])
    masked_lma_values = np.ma.masked_where(~pft_mask[:, :], lma_values[0, :, :])
    
    # Calculate average and std values per PFT for chl, lai, and lma
    average_chl_per_pft = []
    std_chl_per_pft = []
    average_lai_per_pft = []
    std_lai_per_pft = []
    average_lma_per_pft = []
    std_lma_per_pft = []
    for pft_category in pft_categories:
        # chl
        pft_chl_values = masked_chl_values[pft_values[:, :] == pft_category]
        average_chl = np.nanmean(pft_chl_values)
        std_chl = np.nanstd(pft_chl_values)
        average_chl_per_pft.append(average_chl)
        std_chl_per_pft.append(std_chl)
        
        # lai
        pft_lai_values = masked_lai_values[pft_values[:, :] == pft_category]
        average_lai = np.nanmean(pft_lai_values)
        std_lai = np.nanstd(pft_lai_values)
        average_lai_per_pft.append(average_lai)
        std_lai_per_pft.append(std_lai)
        
        # lma
        pft_lma_values = masked_lma_values[pft_values[:, :] == pft_category]
        pft_lma_values = pft_lma_values[(~np.isnan(pft_lma_values)) & (pft_lma_values != 0)]
        average_lma = np.nanmean(pft_lma_values)
        std_lma = np.nanstd(pft_lma_values)
        average_lma_per_pft.append(average_lma)
        std_lma_per_pft.append(std_lma)
    
    # Print average and standard deviation values per PFT
    for idx, (avg_chl, std_chl) in enumerate(zip(average_chl_per_pft, std_chl_per_pft)):
        print(f'PFT {pft_categories[idx] -1 }: Average chl value: {avg_chl}, Standard Deviation: {std_chl}')

    for idx, (avg_lai, std_lai) in enumerate(zip(average_lai_per_pft, std_lai_per_pft)):
        print(f'PFT {pft_categories[idx] -1 }: Average lai value: {avg_lai}, Standard Deviation: {std_lai}')

    for idx, (avg_lma, std_lma) in enumerate(zip(average_lma_per_pft, std_lma_per_pft)):
        print(f'PFT {pft_categories[idx]-1}: Average lma value: {avg_lma}, Standard Deviation: {std_lma}')

    # Create new values arrays for chl, lai, and lma
    new_chl_values = np.zeros_like(chl_values[0,:,:])
    new_std_chl_values = np.zeros_like(chl_values[0,:,:])
    new_lai_values = np.zeros_like(lai_values[0,:,:])
    new_std_lai_values = np.zeros_like(lai_values[0,:,:])
    new_lma_values = np.zeros_like(lma_values[0,:,:])
    new_std_lma_values = np.zeros_like(lma_values[0,:,:])

    # Fill new values arrays with averages per PFT
    for idx, pft_category in enumerate(pft_categories):
        mask = (pft_values == pft_category)[:, :]
        new_chl_values[mask] = average_chl_per_pft[idx]
        new_std_chl_values[mask] = std_chl_per_pft[idx]
        new_lai_values[mask] = average_lai_per_pft[idx]
        new_std_lai_values[mask] = std_lai_per_pft[idx]
        new_lma_values[mask] = average_lma_per_pft[idx]
        new_std_lma_values[mask] = std_lma_per_pft[idx]

    # Create plots for all PFTs for chl, lai, and lma
    for trait, values, new_values, new_std_values, label, y_lim, x_lim in zip(
            ['chl', 'lai', 'lma'],
            [masked_chl_values, masked_lai_values, masked_lma_values],
            [new_chl_values, new_lai_values, new_lma_values],
            [new_std_chl_values, new_std_lai_values, new_std_lma_values],
            [r'CHL Value ($\mu$g.cm$^{-2}$)', r'LAI Value (m$^{2}$.m$^{-2}$)', r'LMA Value (g.cm$^{-2}$)'],
            [0.5, 1.0, 800],
            [20, 7.5, 0.03]):
        
        plt.figure(figsize=(8, 6))
        colors = ['g', 'b', 'r']
        legend_labels = []

        for idx, pft_category in enumerate(pft_categories):
            mask = (pft_values == pft_category)[:, :]
            trait_values_for_pft = values[pft_values == pft_category].compressed()
            avg_trait = np.nanmean(trait_values_for_pft)
            std_trait = np.nanstd(trait_values_for_pft)

            plt.hist(trait_values_for_pft, bins=30, density=True, alpha=0.5, color=colors[idx])
            plt.axvline(avg_trait, color=colors[idx], linestyle='dashed', linewidth=1)

            avg_std_text = f'{avg_trait:.2f} ± {std_trait:.2f}'
            legend_labels.append(f'PFT {pft_category}: {avg_std_text}')

        plt.title(f'Combined {trait.upper()} Distribution ({dates_without_times[int(time)]})', fontsize=18)
        plt.xlabel(label, fontsize=16)
        plt.ylabel('Density', fontsize=16)
        plt.ylim(0, y_lim)
        plt.xlim(0, x_lim)
        plt.legend(legend_labels, fontsize=14)
        plt.xticks(fontsize=14)
        plt.yticks(fontsize=14)

        plt.close()

    # Create new xarray DataArrays for chl, lai, and lma
    def create_dataarray(values, time, lat, lon):
        da = xr.DataArray(values[np.newaxis,:,:], dims=('time', 'lat', 'lon'), coords={'time': time, 'lat': lat, 'lon': lon})
        da = da.where(da != 0, np.nan)
        return da

    new_chl_da = create_dataarray(new_chl_values, chl_ds['time'], chl_ds['lat'], chl_ds['lon'])
    new_std_chl_da = create_dataarray(new_std_chl_values, chl_ds['time'], chl_ds['lat'], chl_ds['lon'])

    new_lai_da = create_dataarray(new_lai_values, lai_ds['time'], lai_ds['lat'], lai_ds['lon'])
    new_std_lai_da = create_dataarray(new_std_lai_values, lai_ds['time'], lai_ds['lat'], lai_ds['lon'])

    new_lma_da = create_dataarray(new_lma_values, lma_ds['time'], lma_ds['lat'], lma_ds['lon'])
    new_std_lma_da = create_dataarray(new_std_lma_values, lma_ds['time'], lma_ds['lat'], lma_ds['lon'])

    # Create new xarray Datasets
    new_chl_ds = xr.Dataset({'chl': new_chl_da, 'std': new_std_chl_da})
    new_lai_ds = xr.Dataset({'lwc': new_lai_da, 'std': new_std_lai_da})
    new_lma_ds = xr.Dataset({'lma': new_lma_da, 'std': new_std_lma_da})

    # Save the new datasets to NetCDF files
    new_chl_ds.to_netcdf(f'/net/fluo/data3/data/FluoData1/students/renato/aviris_dangermond/traits/datasets/clima_fit_prescribed_lai/masked_chl_aviris_dangermond_time_{time}.nc')
    new_lai_ds.to_netcdf(f'/net/fluo/data3/data/FluoData1/students/renato/aviris_dangermond/traits/datasets/clima_fit_prescribed_lai/masked_lwc_aviris_dangermond_time_{time}.nc')
    new_lma_ds.to_netcdf(f'/net/fluo/data3/data/FluoData1/students/renato/aviris_dangermond/traits/datasets/clima_fit_prescribed_lai/masked_lma_aviris_dangermond_time_{time}.nc')

    # Informative message: Data processing and saving completed for the current time
    print(f'Data processing and saving completed for time: {time}\n')

    # Close the opened datasets
    chl_ds.close()
    lai_ds.close()
    lma_ds.close()
    pft_ds.close()


Processing data for time: 00
Extracting data from the loaded datasets...
PFT 1: Average chl value: 63.857608795166016, Standard Deviation: 26.526578903198242
PFT 2: Average chl value: 58.02014923095703, Standard Deviation: 26.676130294799805
PFT 3: Average chl value: 76.7792739868164, Standard Deviation: 20.860881805419922
PFT 1: Average lai value: 4.790730953216553, Standard Deviation: 5.423482894897461
PFT 2: Average lai value: 4.412163257598877, Standard Deviation: 5.146036148071289
PFT 3: Average lai value: 7.9902825355529785, Standard Deviation: 6.627671718597412
PFT 1: Average lma value: 0.011311683803796768, Standard Deviation: 0.012094056233763695
PFT 2: Average lma value: 0.009909155778586864, Standard Deviation: 0.011334367096424103
PFT 3: Average lma value: 0.01913439668715, Standard Deviation: 0.014970325864851475
Data processing and saving completed for time: 00

Processing data for time: 01
Extracting data from the loaded datasets...
PFT 1: Average chl value: 60.826168060

In [5]:
from PIL import Image
import os

# Directory containing the PNG images
image_directory = '/net/fluo/data3/data/FluoData1/students/renato/aviris_dangermond/traits/figures/clima_fit_prescribed_lai/'

# Output GIF file name
output_gif = '/net/fluo/data3/data/FluoData1/students/renato/aviris_dangermond/traits/figures/clima_fit_prescribed_lai/combined_lma_histogram.gif'

# List of PNG files in the directory
png_files = [f for f in os.listdir(image_directory) if f.startswith('combined_lma_histogram_time_') and f.endswith('.png')]
png_files.sort()  # Sort the files in ascending order by filename

# Create a list to store image frames
frames = []

# Open and append each PNG image to the frames list
for png_file in png_files:
    image_path = os.path.join(image_directory, png_file)
    img = Image.open(image_path)
    frames.append(img)

# Save the frames as an animated GIF
frames[0].save(output_gif, save_all=True, append_images=frames[1:], duration=500, loop=0)

print(f'GIF saved as {output_gif}')


GIF saved as /net/fluo/data3/data/FluoData1/students/renato/aviris_dangermond/traits/figures/clima_fit_prescribed_lai/combined_lma_histogram.gif


In [11]:
from PIL import Image
import os

# Directory containing the PNG images
image_directory = '/net/fluo/data3/data/FluoData1/students/renato/aviris_dangermond/traits/'

# Output GIF file name
output_gif = 'combined_chl_histogram.gif'

# List of PNG files in the directory
png_files = [f for f in os.listdir(image_directory) if f.startswith('combined_chl_histogram_time_') and f.endswith('.png')]
png_files.sort()  # Sort the files in ascending order by filename

# Create a list to store image frames
frames = []

# Open and append each PNG image to the frames list
for png_file in png_files:
    image_path = os.path.join(image_directory, png_file)
    img = Image.open(image_path)
    frames.append(img)

# Save the frames as an animated GIF
frames[0].save(output_gif, save_all=True, append_images=frames[1:], duration=500, loop=0)

print(f'GIF saved as {output_gif}')


GIF saved as combined_chl_histogram.gif


In [1]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt

# Define the range of times from 00 to 12
times = ['00', '01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12']

# Define PFT categories
pft_categories = [2, 3, 4]
#pft_categories = [1, 2, 3]
colors = ['g', 'b', 'r']  # Colors for PFTs 2, 3, 4

# Initialize accumulators for each trait and PFT
data_accumulator = {trait: {pft: [] for pft in pft_categories} for trait in ['chl', 'lwc', 'lma']}

# Loop over times to aggregate data
for time in times:
    print(f'Processing data for time: {time}')

    # Load datasets for the current time
    chl_ds = xr.open_dataset(f'/net/fluo/data3/data/FluoData1/students/renato/aviris_dangermond/traits/datasets/clima_fit_prescribed_lai/chl_aviris_dangermond_clima_fit_time_{time}_reg.nc')
    lai_ds = xr.open_dataset(f'/net/fluo/data3/data/FluoData1/students/renato/aviris_dangermond/traits/datasets/clima_fit_prescribed_lai/lwc_aviris_dangermond_clima_fit_time_{time}_reg.nc')
    lma_ds = xr.open_dataset(f'/net/fluo/data3/data/FluoData1/students/renato/aviris_dangermond/traits/datasets/clima_fit_prescribed_lai/lma_aviris_dangermond_clima_fit_time_{time}_reg.nc')
    pft_ds = xr.open_dataset(f'../California_Vegetation_WHRTYPE_Dangermond/output_latlon.nc')

    # Extract data
    chl_values = chl_ds['chl'].values.flatten()
    lai_values = lai_ds['lwc'].values.flatten()
    #change units from g.cm-2 to g.m-2
    lma_values = lma_ds['lma'].values.flatten()*100*100
    pft_values = pft_ds['Band1'].values.flatten()

    # Aggregate data for each PFT
    for pft in pft_categories:
        pft_mask = pft_values == pft
        data_accumulator['chl'][pft].extend(chl_values[pft_mask])
        data_accumulator['lwc'][pft].extend(lai_values[pft_mask])
        data_accumulator['lma'][pft].extend(lma_values[pft_mask])

# Function to plot time-averaged histogram per PFT for a given trait
def plot_time_averaged_histogram(data_accumulator, trait, colors, pft_categories):
    plt.figure(figsize=(8, 6))
    legend_labels = []

    for idx, pft in enumerate(pft_categories):
        data = np.array(data_accumulator[trait][pft])
        valid_data = data[~np.isnan(data)]  # Remove NaNs

        # Exclude the extreme values
        min_val = np.nanmin(valid_data)
        max_val = np.nanmax(valid_data)
        data_to_plot = valid_data[(valid_data > min_val) & (valid_data < max_val)]

        # Calculate average and std
        avg, std = np.nanmean(data_to_plot), np.nanstd(data_to_plot)

        # Plot histogram with a log scale if it's LMA
        if trait == 'lma':
            plt.yscale('log')  # Set y-axis to log scale for LMA

        plt.hist(data_to_plot, bins=30, color=colors[idx], alpha=0.5, density=True)
        plt.axvline(avg, color=colors[idx], linestyle='dashed', linewidth=1)

        # Legend without units
        legend_labels.append(f'PFT {pft-1}: {avg:.2f} ± {std:.2f}' if trait == 'lma' else f'PFT {pft-1}: {avg:.2f} ± {std:.2f}')

    # Set units for x-axis label
    unit = 'μg/cm²' if trait == 'chl' else ('mol/m²' if trait == 'lwc' else 'g/m²')
    plt.title(f'Time-Averaged {trait.upper()} Distribution by PFT', fontsize=18)
    plt.xlabel(f'{trait.upper()} Value ({unit})', fontsize=16)
    plt.ylabel('Density', fontsize=16)
    plt.legend(legend_labels, fontsize=14)
    plt.xticks(fontsize=14)
    plt.yticks(fontsize=14)
    plt.savefig(f'/net/fluo/data3/data/FluoData1/students/renato/aviris_dangermond/traits/figures/clima_fit_prescribed_lai/time_averaged_{trait}_distribution_by_pft.png',dpi=300)
    plt.close()

# Plot for each trait
for trait in ['chl', 'lwc', 'lma']:
    plot_time_averaged_histogram(data_accumulator, trait, colors, pft_categories)


Processing data for time: 00
Processing data for time: 01
Processing data for time: 02
Processing data for time: 03
Processing data for time: 04
Processing data for time: 05
Processing data for time: 06
Processing data for time: 07
Processing data for time: 08
Processing data for time: 09
Processing data for time: 10
Processing data for time: 11
Processing data for time: 12


In [2]:
import subprocess

# Define the input and output filenames
input_files = [
    '/net/fluo/data3/data/FluoData1/students/renato/aviris_dangermond/traits/datasets/clima_fit_prescribed_lai/masked_chl_aviris_dangermond_clima_fit.nc',
    '/net/fluo/data3/data/FluoData1/students/renato/aviris_dangermond/traits/datasets/clima_fit_prescribed_lai/masked_lwc_aviris_dangermond_clima_fit.nc',
    '/net/fluo/data3/data/FluoData1/students/renato/aviris_dangermond/traits/datasets/clima_fit_prescribed_lai/masked_lma_aviris_dangermond_clima_fit.nc'
]

output_files = [
    '/net/fluo/data3/data/FluoData1/students/renato/aviris_dangermond/traits/datasets/clima_fit_prescribed_lai/mean_masked_chl_aviris_dangermond_clima_fit.nc',
    '/net/fluo/data3/data/FluoData1/students/renato/aviris_dangermond/traits/datasets/clima_fit_prescribed_lai/mean_masked_lwc_aviris_dangermond_clima_fit.nc',
    '/net/fluo/data3/data/FluoData1/students/renato/aviris_dangermond/traits/datasets/clima_fit_prescribed_lai/mean_masked_lma_aviris_dangermond_clima_fit.nc'
]

variables = ['chl', 'lwc', 'lma']

# Loop through each file and perform the cdo timmean operation
for input_file, output_file, variable in zip(input_files, output_files, variables):
    command = f'cdo timmean -selname,{variable} {input_file} {output_file}'
    print(f'Executing: {command}')
    subprocess.run(command, shell=True, check=True)

print('All operations completed successfully.')


Executing: cdo timmean -selname,chl /net/fluo/data3/data/FluoData1/students/renato/aviris_dangermond/traits/datasets/clima_fit_prescribed_lai/masked_chl_aviris_dangermond_clima_fit.nc /net/fluo/data3/data/FluoData1/students/renato/aviris_dangermond/traits/datasets/clima_fit_prescribed_lai/mean_masked_chl_aviris_dangermond_clima_fit.nc
Executing: cdo timmean -selname,lwc /net/fluo/data3/data/FluoData1/students/renato/aviris_dangermond/traits/datasets/clima_fit_prescribed_lai/masked_lwc_aviris_dangermond_clima_fit.nc /net/fluo/data3/data/FluoData1/students/renato/aviris_dangermond/traits/datasets/clima_fit_prescribed_lai/mean_masked_lwc_aviris_dangermond_clima_fit.nc
Executing: cdo timmean -selname,lma /net/fluo/data3/data/FluoData1/students/renato/aviris_dangermond/traits/datasets/clima_fit_prescribed_lai/masked_lma_aviris_dangermond_clima_fit.nc /net/fluo/data3/data/FluoData1/students/renato/aviris_dangermond/traits/datasets/clima_fit_prescribed_lai/mean_masked_lma_aviris_dangermond_cl